# Summary
This notebook interfaces with my Anki deck to extract which vocab words were already in my Japanese decks.

Fot this to run, the file `japanese6000.csv` should already be present in the directory.


This website was extremely useful: 
https://www.juliensobczak.com/write/2016/12/26/anki-scripting.html

In [1]:
import sqlite3

In [2]:
db_file = "/home/austin/.local/share/Anki2/User 1/collection.anki2"

In [3]:
conn = sqlite3.connect(db_file)
c = conn.cursor()

## Find which note types are in my Japanese deck
My Japanese Deck id was determined to be 1452955218348 (via manual looking)

In [4]:
jap_note_types = set()
c.execute('SELECT nid FROM cards WHERE did is "1452955218348"')
all_note_ids = c.fetchall()
for note_id in all_note_ids:
    c.execute('SELECT mid FROM notes WHERE id = {}'.format(note_id[0]))
    jap_note_types.add(c.fetchone()[0])

In [5]:
jap_note_types

{1722847518,
 1356965744391,
 1356968358402,
 1452394503690,
 1453856798407,
 1470748883844,
 1471914315711}

In [6]:
# Create list of all note types
import json
c.execute("SELECT models FROM col")
all_note_types = c.fetchone()[0]
all_note_types = json.loads(all_note_types)

# Print off note types in the Japanese decks
for jnt in jap_note_types:
    print(jnt, all_note_types[str(jnt)]['name'])

1356968358402 Katakana with stroke diagrams and audio
1470748883844 Jap_Word_card
1356965744391 Hiragana with stroke diagrams and audio
1453856798407 Grammar3_morphology_cloze
1452394503690 Grammar1_whole_sentence_translate
1722847518 Japanese
1471914315711 Kanji_Card


## Find Japanese Cards with note (=model) type 1722847518
This is my existing "2000 Japanese words vocab deck"

In [7]:
vocab_list = []
c.execute('SELECT flds FROM notes WHERE mid is 1722847518')
all_fields = c.fetchall()
for fields_str in all_fields:
    
    fields = fields_str[0].split("\x1f")
    if any(s in fields[0] for s in ["Japanese Core", "Core Step"]):
        vocab_list.append(fields[1])
    else:
        raise RuntimeError(fields)

In [8]:
len(vocab_list)

2000

### Exactly 2000 words!

In [9]:
import pickle as pkl
with open("Existing words.pkl", "wb") as f:
    pkl.dump(vocab_list, f)

# Filter the 6000 words csv
Put only the new words into a file for importing into my Anki deck

In [10]:
import csv
already_have_vocab = set(vocab_list)
rows_to_keep = []
with open("japanese6000.csv") as f:
    reader = csv.reader(f, delimiter="\t")
    next(reader) # Get rid of the header
    
    for row in reader:
        if row[0] not in already_have_vocab:
            rows_to_keep.append(row)

In [11]:
len(rows_to_keep)

4066

In [15]:
# Write the new csv file
with open("japanese6000_new.csv", "w") as f:
    writer = csv.writer(f, delimiter="\t")
    
    for row in rows_to_keep:
        writer.writerow(row)